In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from util import *

from han_SA_myalgorithm import algorithm



In [3]:
timelimit = 60
result=[]
problem_files=[f"stage/STAGE1_{i}.json" for i in range(1, 19)]
problem_files.append(["alg_test_problems_20240429/TEST_K50_1.json", "alg_test_problems_20240429/TEST_K50_2.json", "alg_test_problems_20240429/TEST_K100_1.json","alg_test_problems_20240429/TEST_K100_2.json", "alg_test_problems_20240429/TEST_K200_1.json", "alg_test_problems_20240429/TEST_K200_2.json"])
for problem_file in problem_files:
    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time)

    alg_start_time = time.time()

    exception = None

    solution = None
    try:
        # Run algorithm!
        solution = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)
    except Exception as e:
        exception = f'{e}'
        print(traceback.format_exc())


    alg_end_time = time.time()

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    checked_solution['time'] = alg_end_time - alg_start_time
    checked_solution['timelimit_exception'] = (alg_end_time - alg_start_time) > timelimit + 1 # allowing additional 1 second!
    checked_solution['exception'] = exception

    checked_solution['prob_name'] = prob['name']
    checked_solution['prob_file'] = problem_file

    result.append(checked_solution['avg_cost'])
    print(f"{problem_file}: {checked_solution['avg_cost']}")

SA start
STAGE1_1.json: 3736.2259999999997
SA start
STAGE1_2.json: 3396.2180000000003
SA start
STAGE1_3.json: 3115.4639999999995


KeyboardInterrupt: 

In [13]:
import pandas as pd
df=pd.DataFrame(result, index=(range(1,19)))
df

,0
1,3702.716000
2,3358.838000
3,3084.609000
4,2737.435500
5,2318.971333
6,2547.351000
7,3527.067000
8,3272.438000
9,3076.417500
10,2616.171000


In [14]:
df.sum()

0    59604.544167
dtype: float64

In [ ]:
draw_route_solution(ALL_ORDERS, checked_solution)

In [ ]:
draw_bundle_solution(ALL_ORDERS, ALL_RIDERS, DIST, checked_solution)